Project - 1


In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [3]:
df_amazon = pd.read_csv ("amazon_alexa.tsv", sep="\t")

In [5]:
# Top 10 records
df_amazon.head(10)

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1
5,5,31-Jul-18,Heather Gray Fabric,I received the echo as a gift. I needed anothe...,1
6,3,31-Jul-18,Sandstone Fabric,"Without having a cellphone, I cannot use many ...",1
7,5,31-Jul-18,Charcoal Fabric,I think this is the 5th one I've purchased. I'...,1
8,5,30-Jul-18,Heather Gray Fabric,looks great,1
9,5,30-Jul-18,Heather Gray Fabric,Love it! I’ve listened to songs I haven’t hear...,1


In [6]:
import string
import spacy

from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

In [7]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):

    """This function will accepts a sentence as input and processes the sentence into tokens, performing lemmatization,
    lowercasing, removing stop words and punctuations."""

    # Creating our token object, which is used to create documents with linguistic annotations.
    #mytokens = parser(sentence)
    mytokens = nlp(sentence)
    #print(mytokens)

    # Lemmatizing each token and converting each token into lowercase
    # Note that spaCy uses '-PRON-' as lemma for all personal pronouns lkike me, I etc
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [9]:
# Extract the verified reviews into a list
reviews = list(df_amazon["verified_reviews"])
# Display the total number of reviews
print("Total number of reviews:", len(reviews))

# Show the last 10 reviews
for review in reviews[-10:]:
    print(review)

Total number of reviews: 3150
Barry
 
My three year old loves it.  Good for doing basic household tasks.  We're going to add a few smart outlets and bulbs.
Awesome device wish I bought one ages ago.
love it
Perfect for kids, adults and everyone in between!!
Listening to music, searching locations, checking time, looking up weather. There are many more task to learn. It was simple to set up.
I do love these things, i have them running my entire home, TV, all my lights, my thermostat, my front door dead bolt, the cameras... and i love listening to npr. these things are well worth the value and they really shine once you stat adding smart devices and you can control your entire home like its right out of star trek.... I named mine computer to get the feel... but its amazing what you can do with these devices and smart systems.
Only complaint I have is that the sound quality isn't great. I mostly use it for commands though, and I have another dot hooked up to my stereo system to play music

In [20]:
from sklearn.base import TransformerMixin

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}


def clean_text(text):
    text = str(text)
    return text.strip().lower() # Corrected indentation

In [13]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [14]:
from sklearn.model_selection import train_test_split

X = df_amazon['verified_reviews']
ylabels = df_amazon['feedback']

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)
print(f'X_train dimension: {X_train.shape}')
print(f'y_train dimension: {y_train.shape}')
print(f'X_test dimension: {X_test.shape}')
print(f'y_train dimension: {y_test.shape}')

X_train dimension: (2205,)
y_train dimension: (2205,)
X_test dimension: (945,)
y_train dimension: (945,)


In [21]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7af4459e0370>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7af4df691870>)),
                ('classifier', LogisticRegression())])

In [22]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)
print(pipe.score(X_test, y_test))

# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

0.9174603174603174
Logistic Regression Accuracy: 0.9174603174603174
Logistic Regression Precision: 0.9253567508232712
Logistic Regression Recall: 0.9882766705744431


In [23]:
import string
import spacy

from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def test(sentence):

    """This function will accepts a sentence as input and processes the sentence into tokens, performing lemmatization,
    lowercasing, removing stop words and punctuations."""

    # Creating our token object, which is used to create documents with linguistic annotations.
    #mytokens = parser(sentence)
    mytokens = nlp(sentence)
    print(mytokens)

    # Lemmatizing each token and converting each token into lowercase
    # Note that spaCy uses '-PRON-' as lemma for all personal pronouns lkike me, I etc
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    print(mytokens)
    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    #print(mytokens)
    # return preprocessed list of tokens
    return mytokens

In [24]:
test('This is the sentence to work')

This is the sentence to work
['this', 'be', 'the', 'sentence', 'to', 'work']


['sentence', 'work']

In [25]:
from sklearn.base import TransformerMixin

# Custom transformer using spaCy
class predictors2(TransformerMixin):
    def transform(self, X, **transform_params):

        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):

        return self


# Basic function to clean the text
def clean_text(text):
    text = str(text)
    #print(text.strip().lower())
    return text.strip().lower()

In [26]:
clean_text('This is the 4 sentence ')

'this is the 4 sentence'

In [27]:
pipe2 = Pipeline([("cleaner", predictors2()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe2.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('cleaner', <__main__.predictors2 object at 0x7af446f6c1c0>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function spacy_tokenizer at 0x7af4df691870>)),
                ('classifier', LogisticRegression())])

In [28]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)
print(pipe.score(X_test, y_test))


# Model Accuracy
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

0.9174603174603174
Logistic Regression Accuracy: 0.9174603174603174
Logistic Regression Precision: 0.9253567508232712
Logistic Regression Recall: 0.9882766705744431
